Basic Python imports

In [8]:
"""Basic Python Imports"""


import numpy as np
from sklearn.linear_model import Ridge

Discrepancy computation code

In [9]:
"Discrepancy computation"


def compute_disc(xs, ys, xt, yt, outer_iters=100, inner_iters=1000, tol=1e-5, lr=0.1):
    """Given features and labels from source (xs, ys) and target (xt, yt), the function outputs the computed discrepancy between the two domains.
    The DC programming method is used to approximate the discrepancy.

    Arguments:
      xs: source feature data of size m x d
      ys: source label data of size m x 1
      xt: target feature data of size n x d
      yt: target label data of size n x 1
      outer_iters: # outer iterations of DC programming.
      inner_iters: # inner iterations of gradient descent for each step of DC programming.
      tol: the objective function improvement tolerance value for the inner loop.
      lr: learning rate for the inner loop
    """
    m = xs.shape[0]
    n = xt.shape[0]
    d = xs.shape[1]

    w = np.random.normal(size=(d, 1))
    w /= np.linalg.norm(w)

    outer_obj_val = np.inf

    loss = []

    for i in range(outer_iters):
        w0 = w
        w = np.random.normal(size=(d, 1))
        w /= np.linalg.norm(w)
        ypred1 = np.matmul(xs, w0)
        ypred2 = np.matmul(xt, w0)

        curr_obj_val = (
            np.linalg.norm(ypred1 - ys) ** 2 / m - np.linalg.norm(ypred2 - yt) ** 2 / n
        )
        loss.append(curr_obj_val)
        if abs(curr_obj_val - outer_obj_val) <= tol:
            return curr_obj_val
        outer_obj_val = curr_obj_val

        """ optimize for w """
        inner_obj_val = np.inf
        for j in range(inner_iters):
            ypred = (np.matmul(xs, w) - ys).squeeze()
            M = np.matmul(np.diag(ypred), xs)
            grad = np.sum(M, axis=0) / m
            grad = np.expand_dims(grad, axis=1)

            ypred2 = (np.matmul(xt, w0) - yt).squeeze()
            M = np.matmul(np.diag(ypred2), xt)
            grad2 = np.sum(M, axis=0) / n
            grad2 = np.expand_dims(grad2, axis=1)

            grad -= grad2

            w -= lr * grad
            if np.linalg.norm(w) > 1:
                w /= np.linalg.norm(w)

            ypred1 = np.matmul(xs, w)
            ypred2 = np.matmul(xt, w0)
            ypred3 = np.matmul(xt, w)

            curr_obj_val = (
                0.5 * np.linalg.norm(ypred1 - ys) ** 2 / m
                - np.sum(np.multiply(ypred2, ypred3)) / n
            )
            if abs(curr_obj_val - inner_obj_val) <= tol:
                break
            inner_obj_val = curr_obj_val

    return -outer_obj_val


def get_dbars(xs, ys, xt, yt, ms):
    """Given data from multiple sources and a target domain, returns a list of discrepancy values between each source and the target.

    Arguments:
      xs: source data of size m x d, where m is the total number of points from all the sources
      ys: source data of size m x 1, where m is the total number of points from all the sources
      xt: target feature data of size n x d
      yt: target label data of size n x 1
      ms: a list containing the source data and target data sizes
    """
    dbars_drift = []
    for t in range(len(ms) - 1):
        n_t = int(np.sum(ms[:t]))  # From defintion of n_t
        t_disc = compute_disc(xs[n_t : n_t + ms[t]], ys[n_t : n_t + ms[t]], xt, yt)
        dbars_drift.append(abs(t_disc))
    return dbars_drift

Main alternate minimization procedure

In [7]:
def altmin(
    xs,
    xt,
    ys,
    yt,
    x_trgt_test,
    y_trgt_test,
    x_trgt_val,
    y_trgt_val,
    lambda_1,
    lambda_2,
    lambda_3,
    dbar,
    p0,
    lr,
    ms,
    maxiters=100,
    niters=1000,
    q_init=None,
    tol=1e-3,
):
    """Given data from multiple sources and a target domain, the procedure runs the alternate minimization algorithm to find the best fit model for the
    target data and reports the resulting test error.

    Arguments:
      xs: source feature data of size m x d
      ys: source label data of size m x 1
      xt: target feature data of size n x d
      yt: target label data of size n x 1
      x_trgt_test: target feature test data
      y_trgt_test: target label test data
      x_trgt_val: target feature validation data
      y_trgt_val: target label validation data
      lambda_1: regularizer term for the infinity norm of q
      lambda_2: regularizer term for the distance from starting point (p0)
      lambda_3: regularizer term for the squared l2 norm of q
      dbar: a list of discrepancy values between each source and target
      p0: the starting distribution over the data points
      lr: the learning rate for gradient descent
      ms: a list containing the source data and target data sizes
      maxiters: the maximum number of iterations of alternate minimization
      niters: the maximum number of iterations of the q optimization step
      q_init: the initial value of q
      tol: the objective function tolerance value for stopping criteria.
    """
    print("Calling altmin")
    """ initialize q """
    m = xs.shape[0]
    n = xt.shape[0]
    x = np.vstack((xs, xt))
    y = np.vstack((ys, yt))

    if q_init is None:
        q = np.random.uniform(size=m + n)
        q = -np.log(q)
        q /= np.sum(q)
    else:
        q = q_init

    loss = []
    prev_obj = np.inf
    T = 0
    best_h = []

    for i in range(maxiters):
        indices = []
        for j in range(m + n):
            if q[j] > 0:
                indices.append(j)
        clf = Ridge(alpha=lambda_1 * np.max(q), solver="cholesky")
        clf.fit(x[indices, :], y[indices], sample_weight=q[indices])

        curr_h = clf.coef_
        ypred = clf.predict(x)

        """ get current objective value """
        curr_obj = compute_obj_val(
            xs,
            xt,
            ys,
            yt,
            ypred,
            q,
            lambda_1,
            lambda_2,
            lambda_3,
            p0,
            dbar,
            np.linalg.norm(curr_h) ** 2,
            ms,
        )
        print(curr_obj)
        if abs(curr_obj - prev_obj) <= tol:
            best_h.append(clf)
            T += 1
        else:
            T = 0

        if i == maxiters - 1:
            best_h.append(clf)

        prev_obj = curr_obj
        if T == 5:
            break

        loss.append(curr_obj)

        """ Update q via gradient descent """
        init_lr = lr
        best_inner_obj = curr_obj
        best_q = q
        iloss = ((y - ypred) ** 2).squeeze()
        for j in range(niters):
            grad = get_gradient(
                xs,
                xt,
                q,
                iloss,
                lambda_1,
                lambda_2,
                lambda_3,
                p0,
                np.linalg.norm(curr_h) ** 2,
                dbar,
                ms,
            )
            grad /= np.linalg.norm(grad)
            q = q - init_lr * grad / (j + 1)
            q = project_simplex(q)
            obj = compute_obj_val(
                xs,
                xt,
                ys,
                yt,
                ypred,
                q,
                lambda_1,
                lambda_2,
                lambda_3,
                p0,
                dbar,
                np.linalg.norm(curr_h) ** 2,
                ms,
            )
            if obj < best_inner_obj:
                best_inner_obj = obj
                best_q = q

        q = best_q
    indices = []
    for j in range(m + n):
        if q[j] > 0:
            indices.append(j)
    clf = Ridge(solver="cholesky")
    clf.fit(x[indices, :], y[indices], sample_weight=q[indices])

    best_h.append(clf)

    """ use val data for model selection """
    best_error = np.inf
    best_model = []
    for clf in best_h:
        ypred = clf.predict(x_trgt_val)
        error = np.linalg.norm(y_trgt_val - ypred) / len(ypred)
        if error < best_error:
            best_error = error
            best_model = clf

    ypred = best_model.predict(x_trgt_test)
    test_error = np.linalg.norm(y_trgt_test - ypred) ** 2 / len(ypred)
    print("test error = {}".format(test_error))

    return test_error, q


def compute_obj_val(
    xs, xt, ys, yt, ypred, q, lambda_1, lambda_2, lambda_3, p0, dbars, hnorm, ms
):
    """Helper function. Computes the objective value of a given solution.

    Arguments:
      xs: source feature data of size m x d
      ys: source label data of size m x 1
      xt: target feature data of size n x d
      yt: target label data of size n x 1
      ypred: model predictions on all the data
      q: current solution
      lambda_1: regularizer term for the infinity norm of q
      lambda_2: regularizer term for the distance from starting point (p0)
      lambda_3: regularizer term for the squared l2 norm of q
      p0: the starting distribution over the data points
      dbars: a list of discrepancy values between each source and target
      hnorm: the squared l2 norm of the Ridge regression predictor
      ms: a list containing the source data and target data sizes
    """
    loss = 0

    y = np.vstack((ys, yt))
    for i in range(len(q)):
        loss += (y[i] - ypred[i]) ** 2 * q[i]

    loss += lambda_3 * np.linalg.norm(q) ** 2

    loss += lambda_2 * np.linalg.norm(q - p0, ord=1)

    loss += lambda_1 * hnorm * np.max(q)

    for t in range(len(ms) - 1):
        # Compute q_t bar
        n_t = int(np.sum(ms[:t]))  # From defintion of n_t
        q_t_bar = np.sum(q[n_t : n_t + ms[t]])
        loss += dbars[t] * q_t_bar

    return loss


def get_gradient(xs, xt, q, loss, lambda_1, lambda_2, lambda_3, p0, hnorm, dbars, ms):
    """Helper function. Computes the gradient of the objective value of a given solution with respoect to the q variable.

    Arguments:
      xs: source feature data of size m x d
      xt: target feature data of size n x d
      q: current solution
      loss: model loss values on all the data
      lambda_1: regularizer term for the infinity norm of q
      lambda_2: regularizer term for the distance from starting point (p0)
      lambda_3: regularizer term for the squared l2 norm of q
      p0: the starting distribution over the data points
      hnorm: the squared l2 norm of the Ridge regression predictor
      dbars: a list of discrepancy values between each source and target
      ms: a list containing the source data and target data sizes
    """
    m = xs.shape[0]
    n = xt.shape[0]

    """ get loss part of gradient """
    loss_grad = loss

    """ get dbar part of gradient """
    dbar_grad = np.zeros((np.sum(ms)))
    for t in range(len(ms) - 1):
        n_t = int(np.sum(ms[:t]))  # From defintion of n_t
        dbar_grad[n_t : n_t + ms[t]] = dbars[t] * np.ones(ms[t])

    """ get lambda_1 part of gradient """
    lambda_1_grad = np.zeros((m + n))
    i1 = np.argmax(q)
    lambda_1_grad[i1] = lambda_1 * hnorm

    """ get lambda_2 part of gradient """
    lambda_2_grad = np.zeros((m + n))
    lambda_2_grad = lambda_2 * np.sign(q - p0)

    """ get lambda_3 part of gradient """
    lambda_3_grad = np.zeros((m + n))
    lambda_3_grad = 2 * lambda_3 * q

    return np.asarray(
        loss_grad + dbar_grad + lambda_1_grad + lambda_2_grad + lambda_3_grad,
        np.float32,
    )


def project_simplex(q):
    """Projects a given vector onto the unit simplex."""
    d = q.shape[0]

    qprime = -np.sort(-q)

    K = 1
    Sum = []
    Sum.append(qprime[K - 1])
    for i in range(2, d + 1):
        Sum.append(Sum[-1] + qprime[i - 1])
        if (Sum[-1] - 1) / i < qprime[i - 1]:
            K = i

    tau = (Sum[K - 1] - 1) / K
    q = q - tau
    for i in range(d):
        q[i] = max(q[i], 0)

    q /= np.sum(q)

    return q

Sample Usage

In [16]:
def generate_simulated_data(
    d, ms, eps1, eps2, alpha=0.5, sigma=0.1, random_w=False, normalize_w_t=True
):
    """Generates simulated data for D_1 to D_{T+1} Gaussian distributions.

    Parameters:
    d (int): Dimension of the data.
    ms (int list): Sample size for each of the T+1 distributions.
    eps1 (float list): Standard deviation for each of the T+1 distributions.
    eps2 (float list): Measures difference between w for D_1 to D_T (has size T).
    alpha (float): Fraction of each D_1 to D_T to leave without noise.
    sigma (float): Standard deviation of noise to add to labels.

    Returns:
    x_d1T_train: Combined X for each of the D_1 to D_T distributions
    y_d1T_train: Combined Y for each of the D_1 to D_T distributions
    x_trgt_train: X train for the D_{T+1} distribution
    y_trgt_train Y train for the D_{T+1} distribution
    x_trgt_test: X test for the D_{T+1} distribution
    y_trgt_test: Y test for the D_{T+1} distribution
    x_trgt_val: X validation for the D_{T+1} distribution
    y_trgt_val: Y validation for the D_{T+1} distribution
    """
    T = len(ms) - 1
    m1T_sum = np.sum(ms[:T])

    # Will hold X for each of D_1 to D_T
    x_d1T_train = []

    for i in range(T):
        x_d1T_train.append(
            np.array(np.random.normal(scale=eps1[i], size=(ms[i], d)), np.float32)
        )

    x_trgt_train = np.array(
        np.random.normal(scale=eps1[T], size=(ms[T], d)), np.float32
    )

    x_trgt_test = np.array(
        np.random.normal(scale=eps1[T], size=(int(10 * m1T_sum), d)), np.float32
    )
    x_trgt_val = np.array(np.random.normal(scale=[T], size=(100, d)), np.float32)

    ws = []
    w_trgt = np.random.normal(size=(d, 1))
    w_trgt /= np.linalg.norm(w_trgt)
    w = np.random.normal(size=(d, 1))
    w /= np.linalg.norm(w)
    ws.append(w_trgt)
    for i in range(T):
        if random_w:
            w = np.random.normal(size=(d, 1))
        w_i = w_trgt + eps2[i] * w
        if normalize_w_t:
            w_i /= np.linalg.norm(w_i)
        ws.insert(-1, w_i)

    y_d1T_train = []
    for i in range(T):
        y_d1T_train.append(
            np.matmul(x_d1T_train[i], ws[i])
            + np.random.normal(scale=sigma, size=(ms[i], 1))
        )
    y_trgt_train = np.matmul(x_trgt_train, ws[T]) + np.random.normal(
        scale=sigma, size=(ms[T], 1)
    )
    y_trgt_test = np.matmul(x_trgt_test, ws[T]) + np.random.normal(
        scale=sigma, size=(10 * m1T_sum, 1)
    )
    y_trgt_val = np.matmul(x_trgt_val, ws[T]) + np.random.normal(
        scale=sigma, size=(100, 1)
    )

    # introduce noise in D_1 to D_T
    for i in range(T):
        for j in range(int(alpha * ms[i]), ms[i]):
            y_d1T_train[i][j] = np.linalg.norm(ws[i]) ** 2
            x_d1T_train[i][j, :] = -100 * ws[i].squeeze()

    # Flatten first T samples
    x_d1T_train = np.array([x for s_t in x_d1T_train for x in s_t])
    y_d1T_train = np.array([y for s_t in y_d1T_train for y in s_t])

    return (
        x_d1T_train,
        y_d1T_train,
        x_trgt_train,
        y_trgt_train,
        x_trgt_test,
        y_trgt_test,
        x_trgt_val,
        y_trgt_val,
    )


# define data generation parameters
num_dims = 10
ms = [100] * 2
standard_devs = [0.1, 10]  # eps1
w_dists_from_trgt = [1]  # eps2

# generate data

xs, ys, xt, yt, x_trgt_test, y_trgt_test, x_trgt_val, y_trgt_val = (
    generate_simulated_data(num_dims, ms, standard_devs, w_dists_from_trgt, alpha=1)
)

# compute discrepancies

dbars = get_dbars(xs, ys, xt, yt, ms)


# set up altmin parameters

lambda_1 = 100
lambda_2 = 0.1
lambda_3 = 1000
lr = 0.01


m = xs.shape[0]
n = xt.shape[0]
p0 = np.zeros(m + n)
p0[m : m + n] = 1.0
p0 /= np.sum(p0)


# call alternate minimization procedure
test_error, q = altmin(
    xs,
    xt,
    ys,
    yt,
    x_trgt_test,
    y_trgt_test,
    x_trgt_val,
    y_trgt_val,
    lambda_1,
    lambda_2,
    lambda_3,
    dbars,
    p0,
    lr,
    ms,
    maxiters=100,
    niters=1000,
    q_init=None,
    tol=1e-3,
)

Calling altmin
[201.85463555]
[78.6488714]
[32.64781729]
[19.80845977]
[15.30804799]
[13.66979927]
[13.45035305]
[13.25518565]
[13.07684972]
[12.91055175]
[12.75548194]
[12.61088836]
[12.47605778]
[12.35032998]
[12.23617096]
[12.13046561]
[12.03190018]
[11.94000018]
[11.85706134]
[11.78219484]
[11.71229916]
[11.6487916]
[11.59145666]
[11.5397616]
[11.49364667]
[11.45222668]
[11.41437175]
[11.37893591]
[11.34618975]
[11.31636598]
[11.28881958]
[11.26309509]
[11.23910999]
[11.21675669]
[11.19591828]
[11.17656088]
[11.15900604]
[11.14292606]
[11.12837265]
[11.11559337]
[11.10444318]
[11.09422969]
[11.08490758]
[11.07626063]
[11.06856207]
[11.06139285]
[11.05470704]
[11.04870922]
[11.0432428]
[11.03829898]
[11.0338459]
[11.02975421]
[11.02591718]
[11.02239359]
[11.01917814]
[11.01629949]
[11.01372727]
[11.01136915]
[11.00928026]
[11.00763046]
[11.00617421]
[11.00494795]
[11.00389801]
[11.00292169]
[11.00207172]
[11.00134098]
[11.00093196]
[11.00084562]
test error = 0.022428921355628146
